# Baseline Modeling for NBA Dataset

In the baseline modeling section we would like to predict the number of points a player will score in a game. We will first start by examine how linear regression model performs when predicting the number of points Lebron James or Giannis Antetokounmpo will score. To do this we will use variables that were created in the Data Wrangling Notebook which give us the average of most of the player stats in the last 5 games. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from scipy import stats
from random import sample
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
games = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/games.csv')
games_details = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/games_details.csv')
players = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/players.csv')
standings = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/nba-games/ranking.csv')

#### importing the data set from the data wrangling notebook

In [2]:
final_boxscore = pd.read_csv('C:/Users/michael.thabane/Documents/Springboard/Capstone2/final_boxscore.csv')

In [3]:
unique_players = players.drop(columns=['TEAM_ID', 'SEASON'])
unique_players.drop_duplicates(inplace = True)

#### Isolating Lebron James career statistics 

In [4]:
unique_players.query('PLAYER_NAME.str.contains("LeBron")', engine='python')

,PLAYER_NAME,PLAYER_ID
199,LeBron James,2544


In [5]:
lebron_stats = final_boxscore.loc[final_boxscore['PLAYER_ID'] == 2544]
lebron_stats

,Unnamed: 0,SEASON,GAME_ID,GAME_DATE_EST,TEAM_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,GAME_LOC,...,FG3AL5,FG3_PCTL5,FTML5,FTAL5,FT_PCTL5,STLL5,PLUS_MINUSL5,TOT_PTSL5,DAYSOFF,YEAR
22,300,2019,21900891,2020-02-29,1610612747,1610612763,1610612747,LAL,Los Angeles,AWAY,...,7.666667,0.353833,4.500000,6.666667,0.597167,0.333333,7.166667,119.833333,1.0,11.0
556,945,2019,21900861,2020-02-25,1610612747,1610612747,1610612740,LAL,Los Angeles,HOME,...,7.166667,0.298833,4.500000,6.500000,0.680500,0.666667,4.500000,118.666667,4.0,11.0
1042,1446,2019,21900842,2020-02-23,1610612747,1610612747,1610612738,LAL,Los Angeles,HOME,...,7.000000,0.343333,4.333333,6.000000,0.694500,0.833333,8.000000,121.166667,2.0,11.0
1359,1772,2019,21900833,2020-02-21,1610612747,1610612747,1610612763,LAL,Los Angeles,HOME,...,7.000000,0.343333,2.833333,4.500000,0.652833,1.166667,12.500000,123.166667,2.0,11.0
1752,2174,2019,21900817,2020-02-12,1610612747,1610612743,1610612747,LAL,Los Angeles,AWAY,...,7.166667,0.382000,2.833333,4.500000,0.652833,1.166667,13.000000,123.000000,9.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566396,575784,2003,20300038,2003-11-01,1610612739,1610612757,1610612739,CLE,Cleveland,AWAY,...,2.000000,0.050000,1.250000,2.500000,0.226000,2.000000,-6.000000,90.250000,4.0,NaN
566659,576047,2003,20300018,2003-10-30,1610612739,1610612756,1610612739,CLE,Cleveland,AWAY,...,1.000000,0.000000,0.333333,1.000000,0.111000,2.333333,-9.000000,91.666667,2.0,NaN
566890,576278,2003,20300014,2003-10-29,1610612739,1610612758,1610612739,CLE,Cleveland,AWAY,...,0.500000,0.000000,0.000000,0.000000,0.000000,1.500000,NaN,91.500000,1.0,NaN
567040,576429,2003,10300011,2003-10-08,1610612739,1610612737,1610612739,CLE,Cleveland,AWAY,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,100.000000,21.0,NaN


Since the 2019 season has not been completed due to the corona virus pandemic we will split the data in which the 2003-2017 seasons will the training set and the 2018 season will be used as the test set.

In [6]:
lebron_test = lebron_stats.loc[lebron_stats['SEASON'] == 2018]
lebron_train = lebron_stats.iloc[120:]

In [7]:
lebron_train_drop = lebron_train.dropna()
y = lebron_train_drop['PTS']
X = lebron_train_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5','FG3_PCTL5','FTML5',
                       'FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5','TOT_PTSL5','DAYSOFF','YEAR']]
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

C:\Users\michael.thabane\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     2.409
Date:                Wed, 24 Jun 2020   Prob (F-statistic):            0.00152
Time:                        22:14:37   Log-Likelihood:                -2901.4
No. Observations:                 849   AIC:                             5837.
Df Residuals:                     832   BIC:                             5918.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           23.4439     11.235      2.087      0.037       1.392      45.496
PTSL5            0.1329      0.342      0.388      0.698      -0.539       0.805
PTSL3            0.0641      0.117      0.547      0.585      -0.166       0.294
MINL5            0.2082      0.116      1.788      0.074      -0.020       0.437
FGML5            0.9391      0.651      1.442      0.150      -0.339       2.217
FGAL5           -0.5390      0.497     -1.084      0.279      -1.515       0.437
FG_PCTL5       -17.3894     17.022     -1.022      0.307     -50.801      16.022
FG3ML5          -1.9574      1.117     -1.752      0.080      -4.150       0.236
FG3AL5           0.6860      0.531      1.291      0.197      -0.357       1.729
FG3_PCTL5       -2.0303      4.100     -0.495      0.621     -10.077       6.016
FTML5            0.2122      0.753      0.282      0.778      -1.266       1.690
FTAL5           -0.0565      0.635     -0.089      0.929      -1.303       1.190
FT_PCTL5        -6.5451      4.751     -1.378      0.169     -15.871       2.780
STLL5           -0.6383      0.479     -1.332      0.183      -1.579       0.302
PLUS_MINUSL5     0.0509      0.051      0.995      0.320      -0.050       0.151
TOT_PTSL5        0.0621      0.056      1.110      0.267      -0.048       0.172
DAYSOFF         -0.1186      0.186     -0.638      0.523      -0.483       0.246
YEAR            -0.0644      0.149     -0.432      0.666      -0.357       0.228
==============================================================================
Omnibus:                       21.030   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.739
Skew:                           0.294   Prob(JB):                     2.58e-06
Kurtosis:                       3.617   Cond. No.                     2.18e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.55e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Export out to Latex file for report

In [8]:
beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

f = open('C:/Users/michael.thabane/Documents/model1result.tex', 'w')
f.write(beginningtex)
f.write(model.summary().as_latex())
f.write(endtex)
f.close()

Our r2 value is very low which means that the variables being used for the model are having a hard time accurately predicting our outcome which is Points scored. Now we will apply the model to the test set to evaluate the accuracy of the model.

#### Creating the constant for test set in order to place the test set into the model

In [9]:
lebron_model_test_drop = lebron_test.dropna()
lebron_model_test_drop = lebron_model_test_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5','FG3_PCTL5','FTML5',
                       'FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5','TOT_PTSL5','DAYSOFF','YEAR']]
lebron_model_test_drop = lebron_model_test_drop.assign(const = 1)
cols = lebron_model_test_drop.columns.tolist()
cols = cols[-1:] + cols[:-1]
lebron_model_test_drop = lebron_model_test_drop[cols]
lebron_model_test_drop

,const,PTSL5,PTSL3,MINL5,FGML5,FGAL5,FG_PCTL5,FG3ML5,FG3AL5,FG3_PCTL5,FTML5,FTAL5,FT_PCTL5,STLL5,PLUS_MINUSL5,TOT_PTSL5,DAYSOFF,YEAR
29077,1,30.333333,29.000000,33.319444,11.333333,23.666667,0.481833,1.333333,5.500000,0.257167,6.333333,8.833333,0.700333,1.000000,5.333333,111.333333,3.0,10.0
29396,1,30.666667,30.750000,32.627778,12.000000,23.666667,0.512167,1.166667,5.666667,0.221500,5.500000,8.666667,0.614667,0.833333,3.166667,109.333333,2.0,10.0
29755,1,31.000000,32.000000,33.577778,12.166667,22.500000,0.542167,1.166667,5.500000,0.225500,5.500000,8.666667,0.614667,0.833333,1.666667,109.166667,2.0,10.0
30701,1,30.000000,31.500000,34.538889,12.000000,21.666667,0.551000,1.833333,6.000000,0.299500,4.166667,7.666667,0.545167,0.833333,0.000000,106.833333,5.0,10.0
31320,1,30.333333,31.000000,35.711111,11.666667,21.000000,0.551667,2.166667,6.500000,0.328667,4.833333,8.166667,0.583333,0.833333,-0.666667,110.500000,3.0,10.0
31508,1,29.833333,28.750000,36.363889,11.333333,21.166667,0.533333,2.500000,7.333333,0.319333,4.666667,8.666667,0.512500,0.833333,-1.833333,110.833333,2.0,10.0
32048,1,28.833333,29.000000,38.358333,10.500000,21.166667,0.497167,2.333333,7.166667,0.286000,5.500000,9.166667,0.574333,1.333333,-3.333333,110.500000,3.0,10.0
32513,1,28.166667,29.500000,39.591667,10.166667,20.333333,0.506500,2.500000,7.000000,0.321667,5.333333,8.833333,0.567333,1.833333,-3.666667,113.166667,3.0,10.0
32884,1,28.500000,28.750000,39.213889,10.500000,21.166667,0.502833,2.333333,6.666667,0.293833,5.166667,8.500000,0.572833,1.833333,-0.833333,114.166667,2.0,10.0
33395,1,28.666667,28.750000,39.594444,10.166667,21.000000,0.490167,2.166667,6.833333,0.269833,6.166667,9.166667,0.637667,2.333333,-0.666667,114.833333,2.0,10.0


In [10]:
Y_pred = model.predict(lebron_model_test_drop)
act = lebron_test.dropna()
Y_actual = act['PTS']

In [11]:
RMSE = np.sqrt(mean_squared_error(Y_actual,Y_pred))
r2 = r2_score(Y_actual,Y_pred)

In [12]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [13]:
MAPE = mean_absolute_percentage_error(Y_actual,Y_pred)

In [14]:
print(' RMSE =', RMSE,'\n','r squared = ', r2,'\n','MAPE = ', MAPE)

 RMSE = 7.753315379958686 
 r squared =  -0.021090252249140296 
 MAPE =  25.842116632470884


We notice that r2 is negative which means the model is performing worse than using the avg points as a predictor. However we are using data from previous season to predict the following season. In practice we know that the year prior may have little to no impact on how the player will score in the season. To have a more realistic test and training split we add half of the games from the 2018 season to the training set and use a smaller test set which contains that later half of games from the 2018 season. Since the model will always be run using all the previous game information you will never run the model as previous. With the new training and test split we will rerun the model and examine how effective it is at predicting points.

In [15]:
lebron_train2 = lebron_stats.iloc[91:]
lebron_test2 = lebron_stats.iloc[61:91]

In [16]:
lebron_train2_drop = lebron_train2.dropna()
y_1 = lebron_train2_drop['PTS']
X_1 = lebron_train2_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5','FG3_PCTL5','FTML5',
                       'FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5','TOT_PTSL5','DAYSOFF','YEAR']]
X_1 = sm.add_constant(X_1)
model2 = sm.OLS(y_1,X_1).fit()
model2.summary()

C:\Users\michael.thabane\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     2.372
Date:                Wed, 24 Jun 2020   Prob (F-statistic):            0.00182
Time:                        22:14:37   Log-Likelihood:                -3001.1
No. Observations:                 876   AIC:                             6036.
Df Residuals:                     859   BIC:                             6117.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           22.8005     10.967      2.079      0.038       1.275      44.326
PTSL5            0.0064      0.330      0.019      0.984      -0.642       0.655
PTSL3            0.1037      0.115      0.905      0.366      -0.121       0.328
MINL5            0.1489      0.111      1.341      0.180      -0.069       0.367
FGML5            0.8834      0.646      1.367      0.172      -0.385       2.152
FGAL5           -0.4422      0.490     -0.902      0.367      -1.404       0.520
FG_PCTL5       -13.8495     16.755     -0.827      0.409     -46.735      19.036
FG3ML5          -1.9930      1.097     -1.817      0.070      -4.145       0.159
FG3AL5           0.8405      0.523      1.608      0.108      -0.185       1.866
FG3_PCTL5       -1.5997      4.031     -0.397      0.692      -9.512       6.313
FTML5            0.2327      0.746      0.312      0.755      -1.232       1.698
FTAL5           -0.0590      0.635     -0.093      0.926      -1.305       1.187
FT_PCTL5        -7.0632      4.758     -1.485      0.138     -16.401       2.275
STLL5           -0.6173      0.477     -1.294      0.196      -1.554       0.319
PLUS_MINUSL5     0.0502      0.050      1.014      0.311      -0.047       0.147
TOT_PTSL5        0.0807      0.053      1.516      0.130      -0.024       0.185
DAYSOFF         -0.0824      0.186     -0.442      0.658      -0.448       0.283
YEAR            -0.1427      0.147     -0.973      0.331      -0.430       0.145
==============================================================================
Omnibus:                       26.116   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.648
Skew:                           0.331   Prob(JB):                     8.14e-08
Kurtosis:                       3.675   Cond. No.                     6.08e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.39e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Export putput to latex file for report

In [17]:
beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

f = open('C:/Users/michael.thabane/Documents/model2result.tex', 'w')
f.write(beginningtex)
f.write(model2.summary().as_latex())
f.write(endtex)
f.close()

Our r2 value is very similar to the first model. Below we will apply the model to the smaller test set to evaluate performance.

In [18]:
lebron_model_test2_drop = lebron_test2.dropna()
lebron_model_test2_drop = lebron_model_test2_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5',
                                                   'FG3_PCTL5','FTML5','FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5',
                                                   'TOT_PTSL5','DAYSOFF','YEAR']]
lebron_model_test2_drop = lebron_model_test2_drop.assign(const = 1)
cols = lebron_model_test2_drop.columns.tolist()
cols = cols[-1:] + cols[:-1]
lebron_model_test2_drop = lebron_model_test2_drop[cols]
lebron_model_test2_drop

,const,PTSL5,PTSL3,MINL5,FGML5,FGAL5,FG_PCTL5,FG3ML5,FG3AL5,FG3_PCTL5,FTML5,FTAL5,FT_PCTL5,STLL5,PLUS_MINUSL5,TOT_PTSL5,DAYSOFF,YEAR
29077,1,30.333333,29.00,33.319444,11.333333,23.666667,0.481833,1.333333,5.500000,0.257167,6.333333,8.833333,0.700333,1.000000,5.333333,111.333333,3.0,10.0
29396,1,30.666667,30.75,32.627778,12.000000,23.666667,0.512167,1.166667,5.666667,0.221500,5.500000,8.666667,0.614667,0.833333,3.166667,109.333333,2.0,10.0
29755,1,31.000000,32.00,33.577778,12.166667,22.500000,0.542167,1.166667,5.500000,0.225500,5.500000,8.666667,0.614667,0.833333,1.666667,109.166667,2.0,10.0
30701,1,30.000000,31.50,34.538889,12.000000,21.666667,0.551000,1.833333,6.000000,0.299500,4.166667,7.666667,0.545167,0.833333,0.000000,106.833333,5.0,10.0
31320,1,30.333333,31.00,35.711111,11.666667,21.000000,0.551667,2.166667,6.500000,0.328667,4.833333,8.166667,0.583333,0.833333,-0.666667,110.500000,3.0,10.0
31508,1,29.833333,28.75,36.363889,11.333333,21.166667,0.533333,2.500000,7.333333,0.319333,4.666667,8.666667,0.512500,0.833333,-1.833333,110.833333,2.0,10.0
32048,1,28.833333,29.00,38.358333,10.500000,21.166667,0.497167,2.333333,7.166667,0.286000,5.500000,9.166667,0.574333,1.333333,-3.333333,110.500000,3.0,10.0
32513,1,28.166667,29.50,39.591667,10.166667,20.333333,0.506500,2.500000,7.000000,0.321667,5.333333,8.833333,0.567333,1.833333,-3.666667,113.166667,3.0,10.0
32884,1,28.500000,28.75,39.213889,10.500000,21.166667,0.502833,2.333333,6.666667,0.293833,5.166667,8.500000,0.572833,1.833333,-0.833333,114.166667,2.0,10.0
33395,1,28.666667,28.75,39.594444,10.166667,21.000000,0.490167,2.166667,6.833333,0.269833,6.166667,9.166667,0.637667,2.333333,-0.666667,114.833333,2.0,10.0


In [19]:
Y_pred2 = model.predict(lebron_model_test2_drop)
Y_actual2 = lebron_test2['PTS']

In [20]:
RMSE_2 = np.sqrt(mean_squared_error(Y_actual2,Y_pred2))
r2_2 = r2_score(Y_actual2,Y_pred2)

In [21]:
MAPE_2 = mean_absolute_percentage_error(Y_actual2,Y_pred2)

In [22]:
print(' RMSE =', RMSE_2,'\n','r squared = ', r2_2,'\n','MAPE = ', MAPE_2)

 RMSE = 5.962926546520658 
 r squared =  0.015025279347450016 
 MAPE =  19.398868355111855


We notice our MAPE improved and went from about 26 percent to 19 percent. Also our r2 is no longer negative but it is still very low. Sicne we are only testing the models performance on Lebron it is better we examine another player to see how effect the model really is. Below we will run the same thing as the second model on Giannis' games dataset but use half of the 2019 season as test and include all seasons from 2013 to half of the 2019 season. 

In [23]:
giannis_stats = final_boxscore.loc[final_boxscore['PLAYER_ID'] == 203507]
giannis_stats

,Unnamed: 0,SEASON,GAME_ID,GAME_DATE_EST,TEAM_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,GAME_LOC,...,FG3AL5,FG3_PCTL5,FTML5,FTAL5,FT_PCTL5,STLL5,PLUS_MINUSL5,TOT_PTSL5,DAYSOFF,YEAR
57,416,2019,21900882,2020-02-28,1610612749,1610612749,1610612760,MIL,Milwaukee,HOME,...,3.166667,0.173833,6.666667,9.166667,0.731000,0.666667,19.333333,118.833333,2.0,7.0
644,1037,2019,21900858,2020-02-25,1610612749,1610612761,1610612749,MIL,Milwaukee,AWAY,...,2.666667,0.257167,7.000000,10.000000,0.700500,0.833333,21.166667,120.833333,3.0,7.0
898,1297,2019,21900851,2020-02-24,1610612749,1610612764,1610612749,MIL,Milwaukee,AWAY,...,3.500000,0.290500,7.500000,11.333333,0.669333,1.000000,19.333333,119.500000,1.0,7.0
1157,1564,2019,21900840,2020-02-22,1610612749,1610612749,1610612755,MIL,Milwaukee,HOME,...,3.500000,0.257167,8.000000,12.000000,0.676333,1.166667,14.000000,118.833333,2.0,7.0
1576,1993,2019,21900820,2020-02-20,1610612749,1610612765,1610612749,MIL,Milwaukee,AWAY,...,3.166667,0.257167,7.500000,11.333333,0.665500,0.666667,10.166667,117.166667,2.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237034,241877,2013,11300086,2013-10-21,1610612749,1610612741,1610612749,MIL,Milwaukee,AWAY,...,1.000000,0.083250,3.250000,4.000000,0.675000,1.000000,NaN,87.750000,2.0,1.0
237730,242598,2013,11300053,2013-10-15,1610612749,1610612763,1610612749,MIL,Milwaukee,AWAY,...,1.333333,0.111000,3.000000,4.000000,0.566667,1.000000,NaN,84.000000,6.0,1.0
238085,242964,2013,11300042,2013-10-12,1610612749,1610612749,1610612766,MIL,Milwaukee,HOME,...,2.000000,0.166500,4.500000,6.000000,0.850000,1.500000,NaN,88.000000,3.0,1.0
238391,243273,2013,11300028,2013-10-10,1610612749,1610612750,1610612749,MIL,Milwaukee,AWAY,...,3.000000,0.333000,7.000000,10.000000,0.700000,2.000000,NaN,87.000000,2.0,1.0


In [24]:
giannis_stats.loc[giannis_stats['SEASON'] == 2019]
giannis_train = giannis_stats.iloc[30:]
giannis_test = giannis_stats.iloc[:30]

In [25]:
giannis_train_drop = giannis_train.dropna()
y_3 = giannis_train_drop['PTS']
X_3 = giannis_train_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5','FG3_PCTL5','FTML5',
                       'FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5','TOT_PTSL5','DAYSOFF','YEAR']]
X_3 = sm.add_constant(X_3)
model_ga = sm.OLS(y_3,X_3).fit()
model_ga.summary()

C:\Users\michael.thabane\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    PTS   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     24.46
Date:                Wed, 24 Jun 2020   Prob (F-statistic):           1.38e-51
Time:                        22:14:37   Log-Likelihood:                -1596.4
No. Observations:                 474   AIC:                             3227.
Df Residuals:                     457   BIC:                             3297.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            5.3140      8.507      0.625      0.533     -11.404      22.032
PTSL5           -0.5187      0.469     -1.107      0.269      -1.440       0.402
PTSL3           -0.1119      0.147     -0.759      0.449      -0.402       0.178
MINL5            0.0122      0.141      0.086      0.932      -0.266       0.290
FGML5            0.4552      0.905      0.503      0.615      -1.323       2.233
FGAL5            0.6012      0.565      1.063      0.288      -0.510       1.712
FG_PCTL5        -2.3889     13.554     -0.176      0.860     -29.025      24.248
FG3ML5          -2.3471      2.115     -1.110      0.268      -6.503       1.809
FG3AL5           0.1062      0.776      0.137      0.891      -1.418       1.631
FG3_PCTL5        4.7041      5.061      0.930      0.353      -5.241      14.649
FTML5            0.9181      0.868      1.058      0.291      -0.787       2.623
FTAL5            0.0022      0.597      0.004      0.997      -1.171       1.175
FT_PCTL5         3.0808      4.464      0.690      0.490      -5.691      11.852
STLL5            0.1746      0.731      0.239      0.811      -1.262       1.612
PLUS_MINUSL5     0.0077      0.072      0.107      0.915      -0.134       0.149
TOT_PTSL5       -0.0441      0.065     -0.676      0.500      -0.172       0.084
DAYSOFF         -0.2625      0.273     -0.961      0.337      -0.799       0.274
YEAR             3.9341      0.604      6.515      0.000       2.747       5.121
==============================================================================
Omnibus:                        2.488   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.288   Jarque-Bera (JB):                2.267
Skew:                           0.133   Prob(JB):                        0.322
Kurtosis:                       3.209   Cond. No.                     7.38e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.18e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Export to Latex file for report

In [26]:
beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

f = open('C:/Users/michael.thabane/Documents/modelgaresult.tex', 'w')
f.write(beginningtex)
f.write(model_ga.summary().as_latex())
f.write(endtex)
f.close()

r2 for the model run on giannis data is much higher than the r2 we observed from the models built on Lebron's data. However to examine the performance of the model we will apply it to the testing set.

In [27]:
giannis_model_test_drop = giannis_test.dropna()
giannis_model_test_drop = giannis_model_test_drop[['PTSL5','PTSL3','MINL5','FGML5','FGAL5','FG_PCTL5','FG3ML5','FG3AL5',
                                                   'FG3_PCTL5','FTML5','FTAL5','FT_PCTL5','STLL5','PLUS_MINUSL5',
                                                   'TOT_PTSL5','DAYSOFF','YEAR']]
giannis_model_test_drop = giannis_model_test_drop.assign(const = 1)
cols = giannis_model_test_drop.columns.tolist()
cols = cols[-1:] + cols[:-1]
giannis_model_test_drop = giannis_model_test_drop[cols]
giannis_model_test_drop

,const,PTSL5,PTSL3,MINL5,FGML5,FGAL5,FG_PCTL5,FG3ML5,FG3AL5,FG3_PCTL5,FTML5,FTAL5,FT_PCTL5,STLL5,PLUS_MINUSL5,TOT_PTSL5,DAYSOFF,YEAR
57,1,26.500000,26.25,32.044444,9.500000,18.000000,0.528000,0.833333,3.166667,0.173833,6.666667,9.166667,0.731000,0.666667,19.333333,118.833333,2.0,7.0
644,1,29.000000,26.00,31.213889,10.666667,18.500000,0.586167,0.666667,2.666667,0.257167,7.000000,10.000000,0.700500,0.833333,21.166667,120.833333,3.0,7.0
898,1,30.333333,29.50,32.166667,11.000000,20.000000,0.554333,0.833333,3.500000,0.290500,7.500000,11.333333,0.669333,1.000000,19.333333,119.500000,1.0,7.0
1157,1,30.333333,30.25,32.627778,10.833333,21.666667,0.504500,0.666667,3.500000,0.257167,8.000000,12.000000,0.676333,1.166667,14.000000,118.833333,2.0,7.0
1576,1,29.833333,29.50,32.700000,10.833333,21.000000,0.524333,0.666667,3.166667,0.257167,7.500000,11.333333,0.665500,0.666667,10.166667,117.166667,2.0,7.0
2601,1,31.500000,32.75,32.272222,11.833333,21.500000,0.565500,0.833333,4.000000,0.290500,7.000000,11.500000,0.596000,0.833333,12.833333,117.166667,12.0,7.0
2938,1,30.333333,31.25,30.683333,11.666667,20.666667,0.578833,1.000000,3.666667,0.333333,6.000000,10.666667,0.542500,0.833333,11.000000,118.000000,2.0,7.0
3287,1,30.000000,29.75,31.002778,11.500000,21.500000,0.544500,0.833333,4.333333,0.166667,6.166667,11.833333,0.510500,0.666667,5.833333,119.333333,2.0,7.0
3561,1,31.166667,29.50,29.480556,11.833333,20.833333,0.582833,1.166667,4.333333,0.233333,6.333333,11.166667,0.555500,0.500000,10.666667,119.166667,2.0,7.0
3913,1,31.333333,29.75,28.913889,12.166667,20.166667,0.609167,1.333333,4.166667,0.283333,5.666667,10.333333,0.539500,0.333333,15.166667,120.333333,2.0,7.0


In [28]:
Y_pred_ga = model.predict(giannis_model_test_drop)
Y_actual_ga = giannis_test['PTS']

In [29]:
RMSE_ga = np.sqrt(mean_squared_error(Y_actual2,Y_pred2))
r2_ga = r2_score(Y_actual_ga,Y_pred_ga)

In [30]:
MAPE_ga = mean_absolute_percentage_error(Y_actual_ga,Y_pred_ga)

In [31]:
print(' RMSE =', RMSE_ga,'\n','r squared = ', r2_ga,'\n','MAPE = ', MAPE_ga)

 RMSE = 5.962926546520658 
 r squared =  -0.04454537567408745 
 MAPE =  23.254694325171055


From above we notice that r2 is again negative and the MAPE is also 23%. From this testing we can conclude that a linear regression model may not be the best method or technique to predict a players points given historical boxscore data. In the next section we will examine other Machine Learning techiques which may result in a better proforming prediction model.